## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD could come from any datasource, e.g. text files, a database, a JSON file etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependancy graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high level spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="rdd1.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation a RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check : confirm the length of the list (it should be 1000)

In [2]:
data = range(1000)
len(data)

#1000

1000

### Initialize an RDD

To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [4]:
import pyspark
sc = pyspark.SparkContext(master='local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [5]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [6]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [7]:
# count
rdd.count()

1000

In [10]:
# first
rdd.first()

0

In [14]:
# take
rdd.take(9)

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [21]:
# top
rdd.top(3)

[999, 998, 997]

In [27]:
# collect
print(type(rdd.collect()))
rdd.collect()[:20]

<class 'list'>


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

## Map functions

Now that you've working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new ecommerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 by a random number from 0-1.

In [31]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures[:4], len(sales_figures), sales_figures.shape

(array([0.05307809, 0.41981187, 0.85307459, 3.84039381]), 1000, (1000,))

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [32]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(20)

[0.05307809309588574,
 0.41981186850608077,
 0.8530745898926384,
 3.8403938050027824,
 1.551623465880469,
 3.302653578924413,
 4.770010910199214,
 6.912270203632262,
 1.9619830165426952,
 7.49693122362617,
 5.26647924596434,
 11.793404629224398,
 3.6391698668472054,
 10.127191561654039,
 5.914147591920155,
 14.708178339093006,
 4.316274354645162,
 16.759353588695333,
 16.22485562000404,
 4.171278022151368]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [33]:
def sales_tax(num):
    return 0.92*num

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call use an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [34]:
# perform action to retrieve rdd values
revenue_minus_tax.take(20)

[0.04883184564821488,
 0.3862269190255943,
 0.7848286227012273,
 3.53316230060256,
 1.4274935886100315,
 3.03844129261046,
 4.388410037383277,
 6.359288587341681,
 1.8050243752192796,
 6.8971767257360765,
 4.845160906287193,
 10.849932258886446,
 3.3480362774994292,
 9.317016236721717,
 5.4410157845665434,
 13.531524071965567,
 3.970972406273549,
 15.418605301599706,
 14.926867170403716,
 3.8375757803792587]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [35]:
discounted = price_items.map(lambda x: 0.9 * x)

In [36]:
discounted.take(10)

[0.04777028378629716,
 0.3778306816554727,
 0.7677671309033746,
 3.4563544245025044,
 1.3964611192924221,
 2.972388221031972,
 4.293009819179293,
 6.221043183269035,
 1.7657847148884258,
 6.747238101263553]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [65]:
discounted_taxed = price_items.map(lambda x: 0.9 * x).map(sales_tax)

In [67]:
discounted_taxed.top(15)

[772.2341666826111,
 758.3079080144475,
 742.0469600208294,
 720.8924307440476,
 716.0373018307431,
 714.1161016970622,
 707.186091129475,
 700.7629601375864,
 700.7278367067074,
 700.487987467589,
 694.5192710499526,
 693.2412378737611,
 692.3790719012703,
 690.8964452064233,
 680.7574110951579]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [57]:
discounted.toDebugString()

b'(10) PythonRDD[20] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[15] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [59]:
def original_and_final(price):
    return (price, 0.9 * 0.92 * price)

In [60]:
mapped = price_items.map(original_and_final)
print(mapped.count())
print(mapped.take(10))

1000
[(0.05307809309588574, 0.043948661083393396), (0.41981186850608077, 0.3476042271230349), (0.8530745898926384, 0.7063457604311046), (3.8403938050027824, 3.179846070542304), (1.551623465880469, 1.2847442297490286), (3.302653578924413, 2.7345971633494144), (4.770010910199214, 3.9495690336449494), (6.912270203632262, 5.723359728607513), (1.9619830165426952, 1.6245219376973519), (7.49693122362617, 6.207459053162469)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [61]:
flat_mapped = price_items.flatMap(original_and_final)
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.05307809309588574, 0.043948661083393396, 0.41981186850608077, 0.3476042271230349, 0.8530745898926384, 0.7063457604311046, 3.8403938050027824, 3.179846070542304, 1.551623465880469, 1.2847442297490286]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criteria. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [64]:
# use the filter function
selected_items = price_items.map(lambda x: 0.9 * x).map(sales_tax).filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.sum()

123000.12947225547

## Reduce

Now it's time to figure out how much money BuyStuff would make from selling one of all of it's items after they've reduced their inventory. Use a reduce method with a lambda function to to add up all of the values in the RDD. Your lambda function should have two variables.

In [79]:
selected_items.reduce(lambda x, y: x + y )  #### UMMM what is this and whyy~!~

123000.12947225547

The time has come for BuyStuff to open up shop and start selling it's goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [80]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(19, 315.13467509589066),
 (19, 328.77719081921646),
 (1, 347.2661136792646),
 (31, 340.9821881440296),
 (9, 334.8525747523765),
 (26, 342.3812794854528),
 (18, 369.15302705397045)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from highest spending to least spending.



In [88]:
# calculate how much each user spent
users_totals = sales_data.reduceByKey(lambda x, y: x + y)

In [92]:
users_totals.top(4, key=lambda x: x[1])

[(50, 7111.133843266804),
 (3, 6993.951272285914),
 (36, 6019.846744717547),
 (12, 5382.046028726367)]

In [93]:
# sort the users from highest to lowest spenders
users_totals.sortBy(keyfunc=lambda x: x[1], ascending=False).take(20)

[(50, 7111.133843266804),
 (3, 6993.951272285914),
 (36, 6019.846744717547),
 (12, 5382.046028726367),
 (22, 5324.670705024808),
 (11, 5052.354947163147),
 (30, 5012.099766455478),
 (26, 4979.65507838893),
 (19, 4978.267512309194),
 (10, 4648.087008605762),
 (25, 4364.472098878403),
 (37, 4296.885382527296),
 (42, 4226.48024147076),
 (1, 3872.6672878124887),
 (4, 3676.1771686587454),
 (27, 3671.3588285523165),
 (32, 2982.29086407702),
 (47, 2910.7773446876554),
 (17, 2880.338972093219),
 (40, 2830.797212815872)]

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [133]:
sorted(sales_data.countByKey().items(), key=lambda x: x[1], reverse=True)

[(1, 14),
 (7, 12),
 (22, 10),
 (28, 10),
 (49, 10),
 (40, 9),
 (37, 9),
 (46, 8),
 (43, 8),
 (11, 7),
 (20, 7),
 (10, 7),
 (25, 6),
 (36, 6),
 (34, 6),
 (18, 6),
 (44, 6),
 (19, 6),
 (47, 6),
 (3, 5),
 (29, 5),
 (12, 5),
 (13, 5),
 (26, 5),
 (8, 5),
 (27, 5),
 (24, 5),
 (42, 5),
 (35, 4),
 (14, 4),
 (2, 4),
 (6, 4),
 (5, 4),
 (17, 4),
 (48, 4),
 (32, 3),
 (41, 3),
 (16, 3),
 (23, 3),
 (9, 3),
 (45, 3),
 (38, 3),
 (15, 2),
 (21, 2),
 (4, 2),
 (30, 2),
 (50, 2),
 (39, 1),
 (33, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
